* random cat data
* random cat data corellated with other cat data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df = pd.DataFrame()

In [25]:
n = 300

def create_new_cat_row(df, feature_name, cat_names, probs):
    df[feature_name] = np.random.choice(cat_names, n, p=probs)
    return df

def create_new_num_row(df, params_list, f_name):
    df[f_name] = np.random.normal(params_list[0], params_list[1], n)
    return df

def get_cat_feature_counts(df, corr_feature, cats):
    counts = {}
    for cat in cats:
        count = len(df.loc[df[corr_feature] == cat])
        counts[cat] = count
    return counts

In [26]:
# input from browser uncorrelated  CAT feature 1
feature_name = 'gender'
cats = ['male', 'female']
params = [0.45, 0.55]
df = create_new_cat_row(df, feature_name, cats, params)

In [9]:
# input from uncorrelated NUM feature 1
feature_name = 'reaction'
params = [2, 0.1]
df = create_new_num_row(df, params, feature_name)

In [10]:
# input from browser uncorrelated CAT feature 2 
fea7re_name = 'handed'
cats = ['left', 'right']
probs = [0.3, 0.7]
df = create_new_cat_row(df, feature_name, cats, probs)

In [27]:
# new CAT feature which is correlated to another one 1
orig_f = 'gender'
new_f_name = 'event'
cats = ['male', 'female']
counts = get_cat_feature_counts(df, orig_f, cats)
new_cats = {'male': {'cats': ['weight lifter', 'ballet'],
                     'prob': [0.7, 0.3],
                     'size': counts['male']},
            'female': {'cats': ['weight lifter', 'ballet'],
                       'prob': [0.3, 0.7],
                       'size': counts['female']}
           }
df = add_corr_cat_feature(df, orig_f, new_f_name, new_cats, True)

__main__:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
# add NUM feature which is correlated to CAT feature 1
orig_f = 'gender'
new_f_name = 'height'
cats = ['male', 'female']
counts = get_cat_feature_counts(df, orig_f, cats)
new_cats = {'male': {'cats': np.nan,
                     'prob': [172, 15],
                     'size': counts['male']},
            'female': {'cats': np.nan,
                       'prob': [152, 5],
                       'size': counts['female']}
           }
df = add_corr_cat_feature(df, orig_f, new_f_name, new_cats, False)

NameError: name 'add_corr_cat_feature' is not defined

In [13]:
# new CAT feature which is correlated to another one 2
orig_f = 'handed'
new_f_name = 'preferred hand'
cats = ['left', 'right']
counts = get_cat_feature_counts(df, feature_name, cats)
new_cats = {'left': {'cats': ['LEFTY', 'm_L'],
                     'prob': [0.3, 0.7],
                     'size': counts['left']},
            'right': {'cats': ['RIGHTY', 'm_R'],
                       'prob': [0.8, 0.2],
                       'size': counts['right']}
           }
df = add_corr_cat_feature(df, orig_f, new_f_name, new_cats, True)

NameError: name 'add_corr_cat_feature' is not defined

In [14]:
# add NUM feature which is correlated to CAT feature 2
orig_f = 'event'
new_f_name = 'weight'
cats = ['weight lifting', 'ballet']
counts = get_cat_feature_counts(df, feature_name, cats)
new_cats = {'weight lifting': {'cats': np.nan,
                     'prob': [50, 1],
                     'size': counts['left']},
            'ballet': {'cats': np.nan,
                       'prob': [10, 1],
                       'size': counts['right']}
           }
df = add_corr_cat_feature(df, orig_f, new_f_name, new_cats, False)

KeyError: 'left'

In [15]:
def combine_rows(row):
    if row[1] is '':
        return row[0]
    else:
        return row[1]

In [16]:
def add_corr_cat_feature(df, orig_f, new_f_name, new_cats, is_cat):
    final_columns = df.columns.to_list()
    final_columns.append(new_f_name)
    final_columns.append('index')
    
    temp_df = df.copy()
    temp_df = temp_df[[orig_f]]

    df['index'] = df.index
    temp_df['index'] = temp_df.index

    temp_df[new_f_name] = np.nan if is_cat else 0
    df[new_f_name] = np.nan if is_cat else 0
    
    for cat in temp_df[orig_f].unique():
        sub_df = temp_df.loc[temp_df[orig_f] == cat]
        sub_dict = new_cats[cat]
        if is_cat:
            sub_df[new_f_name] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))
        else:
            sub_df[new_f_name] = np.random.normal(sub_dict['prob'][0], sub_dict['prob'][1], sub_dict['size'])
            
        temp_df = temp_df.merge(sub_df, on=[orig_f, 'index'], how='left')
        l_suff = new_f_name + '_x'
        r_suff = new_f_name + '_y'
        if is_cat:
            temp_df[new_f_name] = temp_df[[l_suff, r_suff]].apply(lambda x: ''.join(x.dropna()), axis=1)
        else:
            temp_df[new_f_name] = (temp_df[[l_suff, r_suff]]
                                   .fillna(0)
                                   .apply(lambda x: x[l_suff] + x[r_suff], axis=1))   
        temp_df = temp_df[[orig_f, new_f_name, 'index']]
        df = df.merge(temp_df, on=[orig_f, 'index'], how='left')
        l_suff = new_f_name + '_x'
        r_suff = new_f_name + '_y'
        df[new_f_name] = df[[l_suff, r_suff]].apply(combine_rows, axis=1)
        df = df[final_columns]
    df = (df.drop('index',axis=1)
          .reset_index(drop=True)
          .sort_index())
    return df

In [17]:
df.loc[df['handed'] == 'right']['weight'].mean()

KeyError: 'handed'

In [18]:
df.head()

,gender,reaction
0,female,right
1,male,right
2,female,right
3,female,left
4,female,right


In [19]:
df.loc[df['gender'] == 'male']['height'].mean()

KeyError: 'height'

# DYNAMIC FILTERING works - THIS IS MUCH BETTER!

attempt at dynamic filtering...

In [28]:
# use the above to create a GENDER -> EVENT df
dff = df.copy()

In [29]:
dff.loc[dff['gender'] == 'female']['event'].value_counts()

ballet           107
weight lifter     48
Name: event, dtype: int64

In [30]:
dff.loc[dff['gender'] == 'male']['event'].value_counts()

weight lifter    98
ballet           47
Name: event, dtype: int64

In [31]:
dff.head()

,gender,event
0,male,weight lifter
1,male,weight lifter
2,female,weight lifter
3,male,weight lifter
4,female,ballet


GENDER -> EVENT --> weight

['gender', 'event', params]

In [131]:
def get_cat_corr_dfs(df, params, f_name, is_num):
    dfs = []
    for row in params:
        orig_df = df.copy()
        for cat in row[0]:
            orig_df = orig_df.loc[orig_df[cat[0]] == cat[1]]
        if is_num:
            orig_df[f_name] = np.random.normal(row[1][0], row[1][1], len(orig_df))
        else:
            orig_df[f_name] = np.array(np.random.choice(row[1][0], len(orig_df), p=row[1][1]))
        dfs.append(orig_df)
    return dfs

In [87]:
# CAT -> NUM
new_f_name = 'height'
params = [
    [[['gender', 'male']], [1.85, 0.2]],
    [[['gender', 'female']], [1.65, 0.2]]
]
is_numerical = True

In [97]:
# CAT CAT -> NUM
new_f_name = 'weight'
params = [
    [[['gender', 'male'], ['event', 'weight lifter']], [103, 10]],
    [[['gender', 'female'], ['event', 'weight lifter']], [63, 8]],
    [[['gender', 'male'], ['event', 'ballet']], [68, 10]],
    [[['gender', 'female'], ['event', 'ballet']], [43, 2]],
]
is_numerical = True

In [115]:
# CAT -> CAT
new_f_name = 'shoe'
params = [
    [[['gender', 'male']], [['large', 'small'], [0.7, 0.3]]],
    [[['gender', 'female']], [['large', 'small'], [0.3, 0.7]]],
]
is_numerical = False

In [125]:
# CAT CAT -> CAT
new_f_name = 'laces'
params = [
    [[['gender', 'male'], ['shoe', 'large']], [['long', 'short'], [0.9, 0.1]]],
    [[['gender', 'male'], ['shoe', 'small']], [['long', 'short'], [0.6, 0.4]]],
    [[['gender', 'female'], ['shoe', 'large']], [['long', 'short'], [0.5, 0.5]]],
    [[['gender', 'female'], ['shoe', 'small']], [['long', 'short'], [0.1, 0.9]]],
]
is_numerical = False

In [126]:
dfs = get_cat_corr_dfs(dff, params, new_f_name, is_numerical)
dff = pd.concat(dfs).sort_index()

In [127]:
dff.head()

,gender,event,height,weight,shoe,laces
0,male,weight lifter,1.745824,100.907938,large,short
1,female,ballet,1.805985,42.971301,small,short
2,male,weight lifter,2.069955,102.081326,large,long
3,female,ballet,1.255179,44.638896,small,short
4,female,ballet,1.453761,44.850255,small,short


In [37]:
# parsed from browser:
cat_fs = ['gender', 'event']
num_params = [[30, 5]
is_numerical = True

# generate params list of lists
def parse_browser_get_params(df, catfs, params, is_num):
    p = []
    rows = []
    for cat_f in catfs:
        row = []
        for cat in df[cat_f].unique():
            row.append([cat_f, cat])
        rows.append(row)
    if len(rows) == 3:
        for i in rows[0]:
            for j in rows[1]:
                for k in rows[2]:
                    p.append([i, j, k])
                    

    elif len(rows) == 2:
        for i in rows[0]:
            for j in rows[1]:
                p.append([i, j])
    p
    return p

In [38]:
temp = parse_browser_get_params(dff, cat_fs, num_params, is_numerical)

In [39]:
temp

[[['gender', 'male'], ['event', 'weight lifter']],
 [['gender', 'male'], ['event', 'ballet']],
 [['gender', 'female'], ['event', 'weight lifter']],
 [['gender', 'female'], ['event', 'ballet']]]

In [36]:
p = []
if len(temp) == 3:
    for i in temp[0]:
        for j in temp[1]:
            for k in temp[2]:
                p.append([i, j, k])

elif len(temp) == 2:
    for i in temp[0]:
        for j in temp[1]:
            p.append([i, j])
p

[[['gender', 'male'], ['event', 'weight lifter']],
 [['gender', 'male'], ['event', 'ballet']],
 [['gender', 'female'], ['event', 'weight lifter']],
 [['gender', 'female'], ['event', 'ballet']]]

In [198]:
end = []
for x in temp:
    for y in x:
        print(y)
        print(x)

['gender', 'male']
[['gender', 'male'], ['gender', 'female']]
['gender', 'female']
[['gender', 'male'], ['gender', 'female']]
['event', 'weight lifter']
[['event', 'weight lifter'], ['event', 'ballet']]
['event', 'ballet']
[['event', 'weight lifter'], ['event', 'ballet']]
[['gender', 'male'], ['gender', 'female']]
[[['gender', 'male'], ['gender', 'female']], [['event', 'weight lifter'], ['event', 'ballet']], [...]]
[['event', 'weight lifter'], ['event', 'ballet']]
[[['gender', 'male'], ['gender', 'female']], [['event', 'weight lifter'], ['event', 'ballet']], [...]]
[[['gender', 'male'], ['gender', 'female']], [['event', 'weight lifter'], ['event', 'ballet']], [...]]
[[['gender', 'male'], ['gender', 'female']], [['event', 'weight lifter'], ['event', 'ballet']], [...]]


In [161]:
z = []
for r in temp:
    print(r)
    for s in r:
        print(s)

[['gender', 'male'], ['gender', 'female']]
['gender', 'male']
['gender', 'female']
[['event', 'weight lifter'], ['event', 'ballet']]
['event', 'weight lifter']
['event', 'ballet']


In [163]:
a = np.array(temp)

In [172]:
a.reshape(2, 2, 2)

array([[['gender', 'male'],
        ['gender', 'female']],

       [['event', 'weight lifter'],
        ['event', 'ballet']]], dtype='<U13')

In [184]:
np.append(a, a.T)   

array(['gender', 'male', 'gender', 'female', 'event', 'weight lifter',
       'event', 'ballet', 'gender', 'event', 'gender', 'event', 'male',
       'weight lifter', 'female', 'ballet'], dtype='<U13')

In [186]:
a

array([[['gender', 'male'],
        ['gender', 'female']],

       [['event', 'weight lifter'],
        ['event', 'ballet']]], dtype='<U13')

In [ ]:

dff.groupby(['gender', 'shoe', 'laces'], as_index=False)['weight'].count()

In [124]:
dff.groupby(['gender', 'shoe'], as_index=False)['weight'].count()

,gender,shoe,weight
0,female,large,49
1,female,small,127
2,male,large,89
3,male,small,35


In [105]:
dff.head(20)

,gender,event,height,weight
0,male,weight lifter,1.745824,100.907938
1,female,ballet,1.805985,42.971301
2,male,weight lifter,2.069955,102.081326
3,female,ballet,1.255179,44.638896
4,female,ballet,1.453761,44.850255
5,male,weight lifter,2.059757,103.740412
6,male,ballet,1.760813,71.978959
7,female,weight lifter,1.951114,66.213639
8,male,weight lifter,1.833990,94.994739
9,female,weight lifter,1.940666,58.848012


In [68]:
final_df.sort_values(df.index)

KeyError: RangeIndex(start=0, stop=300, step=1)

In [62]:
pd.concat(dfs)

,gender,event,weight
0,male,weight lifter,108.162090
2,male,weight lifter,107.240882
5,male,weight lifter,107.295759
8,male,weight lifter,102.933461
21,male,weight lifter,123.287670
...,...,...,...
288,female,ballet,43.962601
292,female,ballet,44.611775
293,female,ballet,40.768605
294,female,ballet,40.607134


In [34]:
# dynamic filter on mult cats then num
new_cats = {'male': {   'weight lifting': {'cats': np.nan,
                                  'prob': [80, 5],
                                  'size': get_cat_feat_counts(df,
                                                              [['gender', 'male'],
                                                              ['event', 'weight lifter']])
                                                                   },
                        'ballet': {'cats': np.nan,
                                   'prob': [50, 5],
                                   'size': get_cat_feat_counts(df, [['gender', 'male'],
                                                                   ['event', 'ballet']])
                                                                    }
                    },
            'female': {   'weight lifting': {'cats': np.nan,
                                  'prob': [50, 1],
                                  'size': get_cat_feat_counts(df, [['gender', 'female'],
                                                                   ['event', 'weight lifter']])
                                                                   },
                        'ballet': {'cats': np.nan,
                                   'prob': [20, 1],
                                   'size': get_cat_feat_counts(df, [['gender', 'female'],
                                                                   ['event', 'ballet']])}
                    },
           }
            

In [35]:
new_cats

{'male': {'weight lifting': {'cats': nan, 'prob': [80, 5], 'size': 85},
  'ballet': {'cats': nan, 'prob': [50, 5], 'size': 39}},
 'female': {'weight lifting': {'cats': nan, 'prob': [50, 1], 'size': 59},
  'ballet': {'cats': nan, 'prob': [20, 1], 'size': 117}}}

In [ ]:
def add_corr_cat_feature(df, orig_f, new_f_name, new_cats, is_cat, final_columns):  
    temp_df = df.copy()
    temp_df = temp_df[[orig_f]]

    df['index'] = df.index
    temp_df['index'] = temp_df.index

    temp_df[new_f_name] = np.nan if is_cat else 0
    df[new_f_name] = np.nan if is_cat else 0
    
    for cat in temp_df[orig_f].unique():
        sub_df = temp_df.loc[temp_df[orig_f] == cat]
        sub_dict = new_cats[cat]
        if is_cat:
            sub_df[new_f_name] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))
        else:
            sub_df[new_f_name] = np.random.normal(sub_dict['prob'][0], sub_dict['prob'][1], sub_dict['size'])
            
        temp_df = temp_df.merge(sub_df, on=[orig_f, 'index'], how='left')
        l_suff = new_f_name + '_x'
        r_suff = new_f_name + '_y'
        if is_cat:
            temp_df[new_f_name] = temp_df[[l_suff, r_suff]].apply(lambda x: ''.join(x.dropna()), axis=1)
        else:
            temp_df[new_f_name] = (temp_df[[l_suff, r_suff]]
                                   .fillna(0)
                                   .apply(lambda x: x[l_suff] + x[r_suff], axis=1))   
        temp_df = temp_df[[orig_f, new_f_name, 'index']]
        df = df.merge(temp_df, on=[orig_f, 'index'], how='left')
        l_suff = new_f_name + '_x'
        r_suff = new_f_name + '_y'
        df[new_f_name] = df[[l_suff, r_suff]].apply(combine_rows, axis=1)
        df = df[final_columns]
    df = (df.drop('index',axis=1)
          .reset_index(drop=True)
          .sort_index())
    return df

In [17]:
df.sort_index()


,gender,job
0,female,nurse
1,male,footballer
2,female,nurse
3,female,lawyer
4,female,nurse
...,...,...
295,male,footballer
296,male,footballer
297,male,footballer
298,male,footballer


In [63]:
test.pop()

def add_new_categorical_feature(df, orig_f, new_f_name, new_cats):
    # copy df and drop orig_f from orig df - will add after correlation
    temp_df = df.copy()
    # df = df.drop(columns=[orig_f])
    df['index'] = df.index
    
    temp_df = temp_df[[orig_f]]
    temp_df['index'] = temp_df.index
    temp_df[new_f_name] = np.nan
    df[new_f_name] = np.nan
    for cat in temp_df[orig_f].unique():
        print(df)
        print(temp_df)
        sub_dict = new_cats[cat]
        sub_df = temp_df.loc[temp_df[orig_f] == cat]
        sub_df['index'] = sub_df.index
        sub_df[new_f_name] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))
        temp_df = temp_df.merge(sub_df, on=[orig_f, 'index'], how='left')
        l_suff = new_f_name + '_x'
        r_suff = new_f_name + '_y'
        df[new_f_name] = temp_df[[l_suff, r_suff]].apply(lambda x: ''.join(x.dropna()), axis=1)
        df = df[[orig_f, new_f_name, 'index']]
        temp_df = temp_df[[orig_f, new_f_name, 'index']]
        print(df)
        print(temp_df)
        # print(df)
        # print(temp_df)
        # df = df.merge(temp_df, on=[orig_f, 'index'])
    return df

'sdsd132123'

In [64]:
test

['sdsd']

In [63]:
test.pop()

def add_new_categorical_feature(df, orig_f, new_f_name, new_cats):
    # copy df and drop orig_f from orig df - will add after correlation
    temp_df = df.copy()
    # df = df.drop(columns=[orig_f])
    df['index'] = df.index
    
    temp_df = temp_df[[orig_f]]
    temp_df['index'] = temp_df.index
    temp_df[new_f_name] = np.nan
    df[new_f_name] = np.nan
    for cat in temp_df[orig_f].unique():
        print(df)
        print(temp_df)
        sub_dict = new_cats[cat]
        sub_df = temp_df.loc[temp_df[orig_f] == cat]
        sub_df['index'] = sub_df.index
        sub_df[new_f_name] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))
        temp_df = temp_df.merge(sub_df, on=[orig_f, 'index'], how='left')
        l_suff = new_f_name + '_x'
        r_suff = new_f_name + '_y'
        df[new_f_name] = temp_df[[l_suff, r_suff]].apply(lambda x: ''.join(x.dropna()), axis=1)
        df = df[[orig_f, new_f_name, 'index']]
        temp_df = temp_df[[orig_f, new_f_name, 'index']]
        print(df)
        print(temp_df)
        # print(df)
        # print(temp_df)
        # df = df.merge(temp_df, on=[orig_f, 'index'])
    return df

'sdsd132123'

In [224]:
# df_up = add_new_categorical_feature(df, corr_feature, 'job', new_cats)


In [256]:
# input from browser
new_f_name = 'job'

In [257]:

orig_columns = df.columns.to_list()
orig_columns.append('index')
temp_df = df.copy()
# df = df.drop(columns=[orig_f])
df['index'] = df.index

temp_df = temp_df[[orig_f]]
temp_df['index'] = temp_df.index

temp_df[new_f_name] = np.nan
df[new_f_name] = np.nan

cat = 'male'
sub_dict = new_cats[cat]


In [230]:
orig_columns



['gender', 'index']

In [258]:

sub_df = temp_df.loc[temp_df[orig_f] == cat]

In [259]:
sub_df[new_f_name] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))

__main__:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [260]:
temp_df = temp_df.merge(sub_df, on=[orig_f, 'index'], how='left')


In [261]:
l_suff = new_f_name + '_x'
r_suff = new_f_name + '_y'
temp_df[new_f_name] = temp_df[[l_suff, r_suff]].apply(lambda x: ''.join(x.dropna()), axis=1)
  

In [262]:
temp_df = temp_df[[orig_f, new_f_name, 'index']]


In [263]:
df.head()

,gender,index,job
0,male,0,NaN
1,female,1,NaN
2,female,2,NaN
3,female,3,NaN
4,female,4,NaN


In [264]:
df = df.merge(temp_df, on=[orig_f, 'index'], how='left')

In [274]:
df.head()

,gender,index,job_x,job_y
0,male,0,NaN,footballer
1,female,1,NaN,
2,female,2,NaN,
3,female,3,NaN,
4,female,4,NaN,


In [277]:
def combine_rows(row):
    if row[1] is '':
        return row[0]
    else:
        return row[1]

In [279]:
l_suff = new_f_name + '_x'
r_suff = new_f_name + '_y'
df[new_f_name] = df[[l_suff, r_suff]].apply(combine_rows, axis=1)

In [280]:
df.head()

,gender,index,job_x,job_y,job
0,male,0,NaN,footballer,footballer
1,female,1,NaN,,NaN
2,female,2,NaN,,NaN
3,female,3,NaN,,NaN
4,female,4,NaN,,NaN


In [281]:
orig_columns.append(new_f_name)

In [282]:
df = df[orig_columns]


In [283]:
df.head()

,gender,index,job
0,male,0,footballer
1,female,1,NaN
2,female,2,NaN
3,female,3,NaN
4,female,4,NaN


In [284]:
cat = 'female'
sub_dict = new_cats[cat]


In [285]:
df.head()

,gender,index,job
0,male,0,footballer
1,female,1,NaN
2,female,2,NaN
3,female,3,NaN
4,female,4,NaN


In [ ]:
df[new_f_name]

In [286]:

cat = 'female'
sub_dict = new_cats[cat]


In [287]:

sub_df = temp_df.loc[temp_df[orig_f] == cat]

In [288]:
sub_df[new_f_name] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))


__main__:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [289]:
temp_df = temp_df.merge(sub_df, on=[orig_f, 'index'], how='left')

In [290]:
l_suff = new_f_name + '_x'
r_suff = new_f_name + '_y'
temp_df[new_f_name] = temp_df[[l_suff, r_suff]].apply(lambda x: ''.join(x.dropna()), axis=1)
  

In [291]:
temp_df = temp_df[[orig_f, new_f_name, 'index']]

In [292]:
df = df.merge(temp_df, on=[orig_f, 'index'], how='left')

In [293]:
l_suff = new_f_name + '_x'
r_suff = new_f_name + '_y'
df[new_f_name] = df[[l_suff, r_suff]].apply(combine_rows, axis=1)

In [295]:
df = df[orig_columns]

In [297]:
df.sample(10)

,gender,index,job
176,male,176,doctor
108,female,108,nurse
31,female,31,nurse
54,female,54,lawyer
225,female,225,lawyer
141,female,141,nurse
51,female,51,lawyer
87,male,87,footballer
299,female,299,nurse
213,male,213,doctor


In [45]:
sub_df[new_f_name] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))



for cat in temp_df[orig_f].unique():
    print(df)
    print(temp_df)
    sub_dict = new_cats[cat]
    sub_df = temp_df.loc[temp_df[orig_f] == cat]
    sub_df['index'] = sub_df.index
    sub_df[new_f_name] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))
    temp_df = temp_df.merge(sub_df, on=[orig_f, 'index'], how='left')
    l_suff = new_f_name + '_x'
    r_suff = new_f_name + '_y'
    df[new_f_name] = temp_df[[l_suff, r_suff]].apply(lambda x: ''.join(x.dropna()), axis=1)
    df = df[[orig_f, new_f_name, 'index']]
    temp_df = temp_df[[orig_f, new_f_name, 'index']]
    print(df)
    print(temp_df)
    # print(df)
    # print(temp_df)
    # df = df.merge(temp_df, on=[orig_f, 'index'])
return df

__main__:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:


for cat in temp_df[orig_f].unique():
    print(df)
    print(temp_df)
    sub_dict = new_cats[cat]
    sub_df = temp_df.loc[temp_df[orig_f] == cat]
    sub_df['index'] = sub_df.index
    sub_df[new_f_name] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))
    temp_df = temp_df.merge(sub_df, on=[orig_f, 'index'], how='left')
    l_suff = new_f_name + '_x'
    r_suff = new_f_name + '_y'
    df[new_f_name] = temp_df[[l_suff, r_suff]].apply(lambda x: ''.join(x.dropna()), axis=1)
    df = df[[orig_f, new_f_name, 'index']]
    temp_df = temp_df[[orig_f, new_f_name, 'index']]
    print(df)
    print(temp_df)
    # print(df)
    # print(temp_df)
    # df = df.merge(temp_df, on=[orig_f, 'index'])
return df

In [216]:
df = df.drop(columns=[orig_f])

In [217]:
temp_df = temp_df[[orig_f]]

In [231]:
sub_dict = new_cats['female']

In [232]:
sub_dict

{'cats': ['nurse', 'lawyer'], 'prob': [0.3, 0.7], 'size': 193}

In [233]:
sub_df = temp_df.loc[temp_df['gender'] == 'female']

In [234]:
sub_df['fsdf'] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))

/var/folders/vh/zc5ymzmx6mx5l09pzzqvzdhm0000gn/T/ipykernel_34384/1602524711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['fsdf'] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))


In [235]:
sub_df.head()

,gender,fsdf
2,female,lawyer
3,female,lawyer
4,female,lawyer
6,female,lawyer
7,female,nurse


In [ ]:
temp_df['index'] = temp_df.index

In [247]:
sub_df['index'] = sub_df.index

/var/folders/vh/zc5ymzmx6mx5l09pzzqvzdhm0000gn/T/ipykernel_34384/2409249459.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['index'] =sub_df.index


In [248]:
temp_df.head()

,gender,fsdf,index
0,male,footballer,0
1,male,doctor,1
2,female,NaN,2
3,female,NaN,3
4,female,NaN,4


In [249]:
sub_df.head()

,gender,fsdf,index
2,female,lawyer,2
3,female,lawyer,3
4,female,lawyer,4
6,female,lawyer,6
7,female,nurse,7


In [262]:
temp_df = temp_df.merge(sub_df,
        on=['gender', 'index'],
        how='left')
        # suffixes=('_x', None)).ffill(axis=1).drop(columns='fsdf_x')

# df['fsdf']=df['fsdf'].astype(int)

In [267]:
temp_df['fsdf'] = temp_df[['fsdf_x', 'fsdf_y']].apply(lambda x: ''.join(x.dropna()), axis=1)

In [271]:
temp_df = temp_df[['gender', 'fsdf']]

In [273]:
df = df.join(temp_df)

In [274]:
df.head()

,gender,fsdf
0,male,footballer
1,male,doctor
2,female,lawyer
3,female,lawyer
4,female,lawyer


In [237]:
temp_df.join(sub_df['fsdf'])

ValueError: columns overlap but no suffix specified: Index(['fsdf'], dtype='object')

In [151]:
temp_df.head()

,gender,fsdf
0,female,NaN
1,female,NaN
2,female,NaN
3,female,NaN
4,male,footballer


In [152]:
sub_dict = new_cats['female']

In [153]:
sub_df = temp_df.loc[temp_df['gender'] == 'female']

In [155]:
sub_df.head()

,gender,fsdf
0,female,NaN
1,female,NaN
2,female,NaN
3,female,NaN
5,female,NaN


In [156]:
sub_df['fsdf'] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))

/var/folders/vh/zc5ymzmx6mx5l09pzzqvzdhm0000gn/T/ipykernel_34384/1602524711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['fsdf'] = np.array(np.random.choice(sub_dict['cats'], sub_dict['size'], p=sub_dict['prob']))


In [230]:
temp_df = temp_df.merge(sub_df['fsdf'], left_index=True, right_index=True, how='left')

In [174]:
sub_df.head()

,gender,fsdf
4,male,footballer
8,male,doctor
10,male,doctor
13,male,footballer
16,male,footballer


In [175]:
df_1.loc[df_1['gender'] == 'male', 'fsdf'] = sub_df.loc[sub_df['gender'] == 'male', 'fsdf']

/var/folders/vh/zc5ymzmx6mx5l09pzzqvzdhm0000gn/T/ipykernel_34384/1666110175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1.loc[df_1['gender'] == 'male', 'fsdf'] = sub_df.loc[sub_df['gender'] == 'male', 'fsdf']


ValueError: shape mismatch: value array of shape (113,) could not be broadcast to indexing result of shape (0,)

In [176]:
df_1.head()

,gender,fsdf
0,female,nurse
1,female,lawyer
2,female,nurse
3,female,lawyer
5,female,lawyer


In [179]:
df.head()

""
0
1
2
3
4


In [177]:
sub_df.head()

,gender,fsdf
4,male,footballer
8,male,doctor
10,male,doctor
13,male,footballer
16,male,footballer


In [ ]:
df_1.merge(df_2, left_index=True, right_index=True)

In [ ]:
# parse input from browser
# def parse_input_cat_corr_cat(df, cat_names, cat_probs, original_f):
#     new_cats = {}
#     for cat in df[original_f].unqiue():
#         new_cats
    

In [115]:
df.loc[df['gender'] == 'male']

,gender
4,male
8,male
10,male
13,male
16,male
...,...
283,male
286,male
289,male
296,male


In [108]:
df['gender'].unique()

array(['female', 'male'], dtype=object)

In [46]:
df.loc[df['gender'] = cats[0]]['

,gender
0,female
1,male
2,male
3,female
4,female


In [15]:
df['norm_2'] = np.random.normal(50, 3, n)